In [79]:
import time 
import pandas as pd
from selenium import webdriver
import json
from pymongo import MongoClient
import pymongo

### Selenium Driver

In [2]:
browser = webdriver.Chrome('C:\\Users\\Yassine\\Documents\\Python_Projects\\Beginner\\basics\\chromedriver.exe')

C:\Users\Yassine\AppData\Local\Temp\ipykernel_9560\701231104.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\\Users\\Yassine\\Documents\\Python_Projects\\Beginner\\basics\\chromedriver.exe')


### Website

In [3]:
browser.get('https://www.airbnb.com/s/Montpellier/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=march&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&query=Montpellier&checkin=2022-04-01&checkout=2022-04-14&source=structured_search_input_header&search_type=autocomplete_click&place_id=ChIJsZ3dJQevthIRAuiUKHRWh60')

# get titles

In [80]:
def get_titles():
    scrap_titles = browser.find_elements_by_xpath('//*[@class="mj1p6c8 dir dir-ltr"]')
    titles = [scrap.text for scrap in scrap_titles]
    all_titles.extend(titles)


### Get Price per Night

In [81]:
def get_price():
    scrap_prices = browser.find_elements_by_xpath('//*[@class="p1qe1cgb dir dir-ltr"]/div/div/div/div/span[@class="_tyxjp1"]')
    prices = [price.text for price in scrap_prices]
    all_prices.extend(prices)

### Scrap Number allowed Guests

In [82]:
def get_guests():
    scrap_nb_guests = browser.find_elements_by_xpath("//div[@class='i1wgresd dir dir-ltr'][1]/span[1]")
    nb_guests = [x.text for x in scrap_nb_guests]
    all_guests.extend(nb_guests)

### Scrap Rooms Disposition

In [83]:
def type_rooms() :
    scrap_type_rooms = browser.find_elements_by_xpath("//div[@class='i1wgresd dir dir-ltr'][1]/span[3]")
    type_rooms = [x.text for x in scrap_type_rooms]
    all_type_rooms.extend(type_rooms)

### Scrap Details Proprety

In [2]:
def get_details_proprety() :
    
    scrap_proprety_details = browser.find_elements_by_xpath('//div[@class="i1wgresd dir dir-ltr"][last()]')
    details = [x.text for x in scrap_proprety_details]
    all_proprety_details.extend(details)

### Scrap Link 

In [1]:
def get_link() : 
    scrap_link = browser.find_elements_by_xpath('//div[@class="cm4lcvy dir dir-ltr"]/a')
    links = [l.get_attribute('href') for l in scrap_link]
    all_links.extend(links)

### Scrap Title Description

In [84]:
def get_title_description() :
    scrap_title_desciption = browser.find_elements_by_xpath("//span[@class='ts5gl90 tl3qa0j t1nzedvd dir dir-ltr']")
    descriptions = [description.text for description in scrap_title_desciption]
    all_description.extend(descriptions)

### Scrap ALL Pages

In [3]:
all_prices = []
all_titles = []
all_description = []
all_type_rooms = []
all_guests = []
all_links = []
all_proprety_details = []
def check_exists_by_xpath(xpath):
    button = browser.find_element_by_xpath(xpath)
    try :
        while button :
            print('True')
            get_titles()
            get_title_description()
            get_guests()
            type_rooms()
            get_details_proprety()
            get_price()
            get_link()
            button.click()
            time.sleep(3)
    except :
        print('Finished')
check_exists_by_xpath('//a[@class = "_1bfat5l"]')

In [95]:
len(all_prices) , len(all_titles) , len(all_guests) , len(all_description) , len(all_type_rooms)

(300, 300, 300, 300, 300)

### Convert Into DataFrame

In [123]:
airbnb = {'Title' : all_titles , 'Description' : all_description , 'Type of Rooms' : all_type_rooms , 'Allowed Guests': all_guests ,'Details Proprety' : all_proprety_details ,'Price per Night' : all_prices,'Link': all_links}

In [ ]:
df = pd.DataFrame(airbnb)

### Data Cleaning

In [125]:
df['Price per Night']= df['Price per Night'].str.replace('€','')
df['Price per Night']= df['Price per Night'].str.replace('$','')
df['Price per Night']= df['Price per Night'].str.replace(',','')
df['Price per Night']= df['Price per Night'].str.replace('From','')
df['Price per Night'] = df['Price per Night'].astype(int)

In [129]:
df['Allowed Guests']= df['Allowed Guests'].str.replace('guests','')
df['Allowed Guests']= df['Allowed Guests'].str.replace('guest','')
df['Allowed Guests']= df['Allowed Guests'].str.replace('room types','')
df['Allowed Guests'] = df['Allowed Guests'].astype(int)

In [136]:
airbnb_api = df.to_dict(orient="records")
with open('airbnb.json','w') as file :
    json.dump(airbnb_api,file)

### Connect to MongoDB

In [134]:
client = MongoClient('localhost', 27017)
db = client['airbnb']

### Insert In MongoDB

In [138]:
with open('airbnb.json') as f:
    data = json.load(f)
    db.Montpellier.insert_many(data)